In [44]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

import xgboost
from sklearn import preprocessing
import glob

In [45]:
#location of the datasets and name of the dependent variable, csv_files and val respectively

path = ".\\datasets\\"
val = 'btc'

In [46]:
def vars(path):
    '''Function to get all the CSV files in a directory
    Arguments: path, the path to the main directory containing the csv files
    Returns: List of paths to the files'''
    return glob.glob(os.path.join(path, "*.csv"))
    

In [47]:
csv_files = vars(path)

dep_path = [f for f in csv_files if f.split("\\")[-1].split('.')[0] == 'btc']

if(len(dep_path) == 0):
    print("Invalid value")
    sys.exit(1)

else: dep_path = dep_path[0]

csv_files.remove(dep_path)

In [48]:
main_data_cols = ['Date', 'Price']

main_df = pd.read_csv(dep_path, usecols= main_data_cols)
main_df.columns = ['Date', val]

In [49]:
for paths in csv_files:
    df = pd.read_csv(paths, usecols = main_data_cols)
    df.columns = ['Date', paths.split("\\")[-1].split(".")[0]]
    main_df = pd.merge(main_df, df, on ="Date")

In [50]:
main_df

,Date,btc,aluminium,copper,crude-oil,gold,natural_gas,sugar,wheat
0,"Jun 15, 2022","22,577.9","2,592.50",4.1635,115.31,"1,819.60",7.420,18.46,"1,050.00"
1,"Jun 14, 2022","22,157.3","2,570.00",4.1565,118.93,"1,813.50",7.189,18.70,"1,050.25"
2,"Jun 13, 2022","22,448.0","2,628.00",4.2115,120.93,"1,831.80",8.609,18.71,"1,071.00"
3,"Jun 10, 2022","29,083.3","2,680.00",4.2945,120.67,"1,875.50",8.850,18.87,"1,070.75"
4,"Jun 09, 2022","30,097.8","2,761.00",4.3810,121.51,"1,852.80",8.963,19.29,"1,071.25"
5,"Jun 08, 2022","30,201.6","2,819.00",4.4545,122.11,"1,856.50",8.699,18.98,"1,074.75"
6,"Jun 07, 2022","31,128.8","2,778.50",4.4355,119.41,"1,852.10",9.293,18.97,"1,071.75"
7,"Jun 06, 2022","31,367.6","2,782.50",4.4350,118.50,"1,843.70",9.322,19.56,"1,093.00"
8,"Jun 01, 2022","29,798.5","2,726.00",4.3285,115.26,"1,848.70",8.696,19.44,"1,041.25"
9,"May 31, 2022","31,793.4","2,787.00",4.2960,114.67,"1,848.40",8.145,19.40,"1,087.50"
